<a href="https://colab.research.google.com/github/Srijani-coder/dambladore-talking/blob/main/Research_Reproduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install openai
import openai
import argparse
import pandas as pd
import openai
from time import sleep
import time
import os
import pdb
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from collections import OrderedDict
import pandas as pd
import regex as re
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("hkust-nlp/felm", 'wk')
print(dataset['test'][0])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/184 [00:00<?, ? examples/s]

{'index': 0, 'prompt': 'Which country or city has the maximum number of nuclear power plants?', 'response': 'The United States has the highest number of nuclear power plants in the world, with 94 operating reactors. Other countries with a significant number of nuclear power plants include France, China, Russia, and South Korea.', 'segmented_response': ['The United States has the highest number of nuclear power plants in the world, with 94 operating reactors.', 'Other countries with a significant number of nuclear power plants include France, China, Russia, and South Korea.'], 'labels': [False, True], 'comment': ['As of December 2022, there were 92 operable nuclear power reactors in the United States.', ''], 'type': ['knowledge_error', None], 'ref': ['https://www.eia.gov/tools/faqs/faq.php?id=207&t=3'], 'source': 'Quora'}


In [8]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("hkust-nlp/felm", 'science')
print(dataset['test'][0])

Generating test split:   0%|          | 0/125 [00:00<?, ? examples/s]

{'index': 0, 'prompt': 'What happens to you if you eat watermelon seeds?', 'response': 'If you accidentally or intentionally swallow a few watermelon seeds while eating the fruit, there is no need to worry. You can safely consume watermelon seeds as they are edible and even possess some beneficial properties such as being a good source of protein, healthy fats, vitamins, and minerals.\\n\\nHowever, eating a large amount of watermelon seeds (in the order of dozens or more) can lead to gastrointestinal issues like bloating, gas, and irritation. This is because watermelon seeds contain compounds such as tannins, which can interfere with digestion and lead to discomfort.\\n\\nAdditionally, some people may be allergic to watermelon seeds, and consuming them could trigger allergic reactions like itching, hives, or difficulty breathing.\\n\\nIn conclusion, eating a small number of watermelon seeds is not harmful and may even be beneficial, but consuming large amounts can cause digestive issue

In [10]:
dataset = load_dataset("hkust-nlp/felm", 'math')
print(dataset['test'][0])

Generating test split:   0%|          | 0/194 [00:00<?, ? examples/s]

{'index': 0, 'prompt': 'Show me the answer for the following math question: 143287*534156=?', 'response': 'To find the product of 143287 and 534156, you can use the standard long multiplication method as follows:\\n\\nmarkdown\\nCopy code\\n143287*534156\\n---------\\n715740   (6 x143287)\\n+4294440   (5 x 143287)\\n+11448176   (4 x 143287)\\n---------\\n76529372\\nTherefore, 143287 multiplied by 534156 equals 76529372.', 'segmented_response': ['To find the product of 143287 and 534156, you can use the standard long multiplication method as follows:\n\n\n\n\n143287*534156\n---------\n715740   (6 x143287)\n+4294440   (5 x 143287)\n+11448176   (4 x 143287)\n---------\n76529372\n\n\n\n\nTherefore, 143287 multiplied by 534156 equals 76529372.'], 'labels': [False], 'comment': ['76537610772'], 'type': [''], 'ref': [''], 'source': 'authors'}


In [11]:
dataset = load_dataset("hkust-nlp/felm", 'reasoning')
print(dataset['test'][0])

Generating test split:   0%|          | 0/208 [00:00<?, ? examples/s]

{'index': 0, 'prompt': 'If you pamper a cow and give it lots of stuff, does it produce spoiled milk?', 'response': "No, pampering a cow and giving it lots of stuff does not necessarily result in spoiled milk. The quality of milk depends on various factors such as the cow's health, hygiene, and the way the milk is handled and stored. Proper care and management of the cow, including a balanced diet, clean and comfortable living conditions, and regular health check-ups, can help ensure the production of high-quality milk. Additionally, proper handling and storage of milk, such as keeping it at the right temperature and avoiding contamination, can also help prevent spoilage.", 'segmented_response': ['No, pampering a cow and giving it lots of stuff does not necessarily result in spoiled milk.', "*The quality of milk depends on various factors such as the cow's health, hygiene, and the way the milk is handled and stored.", '*Proper care and management of the cow, including a balanced diet, c

In [14]:
dataset.save_to_disk("/content/all.jsonl")


Saving the dataset (0/1 shards):   0%|          | 0/208 [00:00<?, ? examples/s]

In [18]:
command = 'python eval.py --path="/content/all.jsonl/dataset_dict.json" --model="gpt-3.5-turbo" --key="sk-proj-68wx3fUV5ENcIXhbsNpMT3BlbkFJoCoKdY2EsgpfashTGu1e" --method="raw"'

In [19]:
!{command}

save: res/day2024_04_17.log/day2024_04_17.log
***********************day2024_04_17************************
Traceback (most recent call last):
  File "/content/eval.py", line 341, in <module>
    result=run(data,args.model,args.method,args.num_cons)
  File "/content/eval.py", line 104, in run
    _id=str(data['index'])
KeyError: 'index'


In [3]:
@retry(wait=wait_random_exponential(min=8, max=50), stop=stop_after_attempt(6))
def gene(eval_prompt,dataname,model,method):

    if model=='vicuna_30B':


        eval_prompt+="\nBelow are your outputs:"
        eval_prompt+="\nAnswers:"
        input_ids = tokenizer(eval_prompt, return_tensors="pt").input_ids.to("cuda")
        outputs = model_.generate(input_ids,max_new_tokens=100)

        return (tokenizer.decode(outputs[0])[len(eval_prompt)+4:],eval_prompt)
    elif model=="gpt-3.5-turbo" or model=="gpt-4":
        prefix=[{"role":"system","content":"You are a helpful assistant."}]
        prefix+= [{"role":"user","content": eval_prompt}]
        if method=='cot_cons':
            request=openai.ChatCompletion.create(
            model=model,
                messages=prefix,
                temperature=0.9,
                max_tokens=100
            )
        else:
            request=openai.ChatCompletion.create(
            model=model,
                messages=prefix,
                temperature=0,
                max_tokens=100
            )
        return (request["choices"][0]['message']['content'],prefix)



In [4]:
result = gene(eval_prompt, dataname, model="gpt-3.5-turbo", method="cot_cons")


NameError: name 'eval_prompt' is not defined